In [39]:
import math 

def read_board(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    color = lines[0].strip()
    if color == 'O':
        color = 1
    else:
        color = -1
    lines = lines[2:]
    size = len(lines)
    board = [[0] * size for _ in range(size)]
    for i in range(size):
        for j in range(size):
            if lines[i][j] == 'O':
                board[i][j] = 1
            elif lines[i][j] == 'X':
                board[i][j] = -1
    return board, color

def check_num_flips(position, color, board):
    num_flips = 0
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]
    for dir in directions:
        dx, dy = dir
        x, y = position
        x += dx
        y += dy
        while 0 <= x < len(board) and 0 <= y < len(board[0]) and board[x][y] == -color:
            x += dx
            y += dy
        if 0 <= x < len(board) and 0 <= y < len(board[0]) and board[x][y] == color:
            while True:
                x -= dx
                y -= dy
                if (x, y) == position:
                    break
                num_flips += 1
    return num_flips
    
def take_step(position, color, board):
    new_board = [row[:] for row in board]
    x, y = position
    new_board[x][y] = color
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]
    for dir in directions:
        dx, dy = dir
        nx, ny = x + dx, y + dy
        while 0 <= nx < len(board) and 0 <= ny < len(board[0]) and board[nx][ny] == -color:
            nx += dx
            ny += dy
        if 0 <= nx < len(board) and 0 <= ny < len(board[0]) and board[nx][ny] == color:
            while (nx, ny) != position:
                nx -= dx
                ny -= dy
                new_board[nx][ny] = color
    return new_board

def check_stable_direction(board, color, position):
    x, y = position
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]
    stable_directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]
    for dir in directions:
        dx, dy = dir
        nx, ny = x + dx, y + dy
        while 0 <= nx < len(board) and 0 <= ny < len(board[0]):
            if board[nx][ny] == color:
                nx += dx
                ny += dy
            else:
                stable_directions.remove(dir)
                break
    return stable_directions

def direction_stable_3(stable_direction):
    stable = [[(1,0),(1,1),(0,1)],[(0,1),(-1,1),(-1,0)],[(-1,0),(-1,-1),(0,-1)],[(0,-1),(1,-1),(1,0)]]
    for direction in stable:
        if all(dir in stable_direction for dir in direction):
            return True
    return False
            

def check_stability(board, color, position):
    x, y = position
    new_board = take_step(position, color, board)
    stable_direction = check_stable_direction(new_board, color, position)
    if (x, y) in [(0, 0), (0, 11), (11, 0), (11, 11)]:
        return True
    elif x == 0 or x == 11:
        if (0, 1) in stable_direction or (0, -1) in stable_direction:
            return True
    elif y == 0 or y == 11:
        if (1, 0) in stable_direction or (-1, 0) in stable_direction:
            return True
    else:
        if direction_stable_3(stable_direction):
            return True
    return False

def get_candidates(board, color, all_positions):
    candidates = []
    for position in all_positions:
        x,y = position
        flips = check_num_flips(position, color, board)
        if flips > 0 and board[x][y] == 0:
            candidates.append(position)
    return candidates

def get_candidates_flips(board, candidates):
    candidate_flips = {}
    for position in candidates:
        flips = check_num_flips(position, color, board)
        if flips > 0:
            candidate_flips[position] = flips
    return candidate_flips

def get_self_mobility(board, color, candidates, all_positions):
    self_mobility = {}
    for candidate in candidates:
        new_board = take_step(candidate, color, board)
        self_mobility[candidate] = len(get_candidates(new_board, color, all_positions))
    return self_mobility

def get_best_self_mobility(board, color, position, all_positions):
    new_board = take_step(position, color, board)
    opponent_candidates = get_candidates(new_board, -color, all_positions)
    all_self = get_opponent_mobility(new_board, -color, opponent_candidates, all_positions)
    best_self = all_self[max(all_self)]
    return best_self

def get_opponent_mobility(board, color, candidates, all_positions):
    opponent_mobility = {}
    for candidate in candidates:
        new_board = take_step(candidate, color, board)
        opponent_mobility[candidate] = len(get_candidates(new_board, -color, all_positions))
    return opponent_mobility

def step2_best_position(board, color, candidates, all_positions):
    best_positions = {}
    for candidate in candidates:
        best_positions[candidate] = get_best_self_mobility(board, color, candidate, all_positions)
    best_position = max(best_positions)
    return best_position

def has_self_mobility(board, color, position, all_positions):
    new_board = take_step(position, color, board)
    opponent_candidates = get_candidates(new_board, -color, all_positions)
    all_self = get_opponent_mobility(new_board, -color, opponent_candidates, all_positions)
    if all_self[min(all_self)]==0:
        return False
    return True

def output_file(output_filename, position):
    x , y = position
    row = ['a','b','c','d','e','f','g','h','i','j','k','l']
    column = ['1','2','3','4','5','6','7','8','9','10','11','12']
    output = row[y] + column[x]
    with open(output_filename, 'w') as file:
        file.write(output)

def get_board_occupancy(board):
    count = 0
    for row in board:
        for i in row:
            if i != 0:
                count = count + 1
    return count

def get_best_move(board, color, candidates, all_positions):
        opponent_mobility = get_opponent_mobility(board, color, candidates, all_positions)
        min_opponent_mobility = min(opponent_mobility.values())
        min_opponent_positions = [key for key, value in opponent_mobility.items() if value == min_opponent_mobility]
        best_move = step2_best_position(board, color, min_opponent_positions, all_positions)
        return best_move

# Example usage:
input_filename = 'testing_input.txt'
board, color = read_board(input_filename)
opponent_color = -color
for row in board:
    print(row)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [40]:
all_positions = []
for i in range(0, len(board)):
    for j in range(0, len(board[0])):
        all_positions.append((i,j))
print(all_positions)


[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (10, 0), (10, 1), (

In [41]:
pos = (1, 3)
new_board = take_step(pos, color, board)
for row in new_board:
    print(row)
x = check_stable_direction(board, color, pos)
print(x)
a = check_stability(board, color, pos)
print(a)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[]
False


In [42]:
candidates = get_candidates(board, color, all_positions)
candidate_flips = get_candidates_flips(board, candidates)
print(candidates)


[(1, 1), (1, 2), (2, 1), (7, 9), (8, 7), (10, 7)]


In [43]:
position = (2,3)
x , y = position
row = ['a','b','c','d','e','f','g','h','i','j','k','l']
column = ['1','2','3','4','5','6','7','8','9','10','11','12']
output = row[y] + column[x]
print(output)

d3


In [44]:
step2_best_position(board, color, candidates, all_positions)

(10, 7)

In [45]:
d = get_self_mobility(board, color, candidates, all_positions)
x = min(d.values())
min_keys = [key for key, value in d.items() if value == x] 
print(min_keys)

o = get_opponent_mobility(board, color, candidates, all_positions)
print(o)


[(1, 1), (1, 2), (2, 1)]
{(1, 1): 3, (1, 2): 3, (2, 1): 3, (7, 9): 9, (8, 7): 8, (10, 7): 8}


In [46]:
output_filename = 'output.txt'
for position in candidates:
    if check_stability(board, color, position) and has_self_mobility(board, color, position, all_positions):
        output_file(output_filename, position)
        break
    elif position[0] not in [1,3,8,10] or position[1] not in [1,3,8,10]:
        output = get_best_move(board, color, candidates, all_positions)
        output_file(output_filename, output)
        break
    elif position[0] not in [1,10] or position[1] not in [1,10]:
        output = get_best_move(board, color, candidates, all_positions)
        output_file(output_filename, output)
        break
    else:
        output = get_best_move(board, color, candidates, all_positions)
        output_file(output_filename, output)
        break
        